# Initializing the data folder

In [2]:
from pathlib import Path
from dotenv import load_dotenv

try:
    start = Path(__file__).resolve()
except NameError:
    start = Path.cwd()

supporting_files = next(p / "00-supporting-files" for p in start.parents if (p / "00-supporting-files").exists())

In [3]:
supporting_files

PosixPath('/Users/progressedd/personal-projects/project-template/00-supporting-files')

## Getting a tree printout of the repository

In [10]:
from pathlib import Path
import os

def get_directory_tree(directory_path, level=0, prefix='', max_depth=None):
    path = Path(directory_path)
    
    # Early return if we've reached max depth
    if max_depth is not None and level >= max_depth:
        # Count files and folders at this level
        items = list(path.iterdir())
        folders = [item for item in items if item.is_dir()]
        files = [item for item in items if item.is_file()]
        
        # Count file types
        file_types = {}
        for file in files:
            ext = file.suffix.lower() or 'no extension'
            file_type = ext[1:] if ext.startswith('.') else ext
            file_types[file_type] = file_types.get(file_type, 0) + 1
        
        # Build summary string
        summary_parts = []
        if folders:
            summary_parts.append(f"{len(folders)} folder{'s' if len(folders) > 1 else ''}")
        
        for file_type, count in sorted(file_types.items(), key=lambda x: x[1], reverse=True):
            summary_parts.append(f"{count} {file_type}{'s' if count > 1 else ''}")
        
        summary = ", ".join(summary_parts)
        return [f"{prefix}├── ... {len(items)} more items: {summary}"]
    
    result = []
    
    # Add directory name for non-root directories
    if level > 0:
        connector = "└──" if "└" in prefix else "├──"
        result.append(f"{prefix}{connector} {path.name}/")
    
    # Process all files and subdirectories
    items = sorted(path.iterdir(), key=lambda x: (x.is_file(), x.name))
    
    for i, item in enumerate(items):
        is_last = (i == len(items) - 1)
        
        # Choose the appropriate prefix for the next level
        if is_last:
            next_prefix = prefix + "    "  # space after last item
            current_connector = "└──"
        else:
            next_prefix = prefix + "│   "  # vertical line for items that have siblings
            current_connector = "├──"
        
        if item.is_file():
            result.append(f"{prefix}{current_connector} {item.name}")
        else:
            # For directories, recursively process
            child_prefix = next_prefix
            child_tree = get_directory_tree(item, level+1, child_prefix, max_depth)
            result.extend(child_tree)
    
    return result

# Example usage
def print_tree(directory, max_depth=None):
    tree = get_directory_tree(directory, max_depth=max_depth)
    for line in tree:
        print(line)

# Use with your supporting_files path
print_tree(current_dir.parents[1], 2)

# Or with depth limit
# print_tree(supporting_files, max_depth=2)

│   ├── .git/
│   │   ├── ... 14 more items: 14 samples
│   │   ├── ... 1 more items: 1 no extension
│   │   ├── ... 2 more items: 1 folder, 1 no extension
│   │   ├── ... 1 more items: 1 folder
│   │   ├── ... 19 more items: 19 folders
│   │   ├── ... 3 more items: 3 folders
│   ├── COMMIT_EDITMSG
│   ├── FETCH_HEAD
│   ├── HEAD
│   ├── config
│   ├── description
│   └── index
│   ├── .venv/
│   │   ├── ... 28 more items: 19 no extensions, 3 bats, 1 13, 1 csh, 1 fish, 1 nu, 1 ps1, 1 py
│   │   ├── ... 1 more items: 1 folder
│   │   ├── ... 1 more items: 1 folder
│   │   ├── ... 2 more items: 2 folders
│   ├── .gitignore
│   ├── CACHEDIR.TAG
│   └── pyvenv.cfg
│   ├── 00-dev-log/
│   └── 00-template.md
│   ├── 00-supporting-files/
│       ├── ... 1 more items: 1 no extension
│   ├── 01-dev-onboarding/
│   │   ├── ... 2 more items: 2 mds
│   │   ├── ... 3 more items: 2 mds, 1 code-profile
│   │   ├── ... 4 more items: 2 tomls, 1 lock, 1 md
│   │   ├── ... 1 more items: 1 folder
│   ├── 

In [12]:
print_tree(supporting_files, 2)

    ├── data/
    └── sample.env.file


In [13]:
files = [file.name for file in Path(supporting_files).iterdir()]
files

['data']

# Loading environment files

In [4]:
from pathlib import Path
from dotenv import load_dotenv

try:
    start = Path(__file__).resolve()
except NameError:
    start = Path.cwd()

supporting_files = next(p / "00-supporting-files" for p in start.parents if (p / "00-supporting-files").exists())
load_dotenv(supporting_files / "data" / "sample.env.file")

True

## Using AzureOpenAI

In [2]:
os.environ["AZURE_OPENAI_ENDPOINT"]

'$INSERT_ENDPOINT'

In [2]:
from openai import OpenAI, AzureOpenAI
client =  AzureOpenAI(
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_version="2024-08-01-preview"
)

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a limerick about the wonders of GPU computing."}
    ]
)
print(completion.choices[0].message.content.strip())

In silicon lanes, data streams fly,  
As GPUs crunch numbers up high.  
With parallel grace,  
They conquer the race,  
Unlocking the secrets, oh my!


## Using NVIDIA NIMs

In [2]:
os.environ["NIMS_BASE_URL"]

'https://integrate.api.nvidia.com/v1'

In [ ]:
from openai import OpenAI

client = OpenAI(
  base_url = os.environ["NIMS_BASE_URL"],
  api_key = os.environ["NIMS_API_KEY"]
)

completion = client.chat.completions.create(
  model="meta/llama-3.3-70b-instruct",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Write a limerick about the wonders of GPU computing."}
  ],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")



There once was a GPU so fine,
Whose computing powers did shine.
It processed with speed,
And its cores did proceed,
To crunch numbers in a divine design.

# Using Structured Outputs With Pydantic and Local LLMs

In [1]:
from pydantic import BaseModel, Field
from typing import List, Dict, Optional

class PoeticResponse(BaseModel):
    User_Instruction: str = Field(..., description="the subject of the user's instruction")
    Poem: str = Field(..., description="the poem version for a given user's instruction")
    Limerick: str = Field(..., description="the limerick version for a user's instruction")
    Rap_Lyric: str = Field(..., description="the rap version for a user's instruction")

class PoeticResponses(BaseModel):
    responses: List[PoeticResponse] = Field(..., description="list of poetic responses if user asks for more than one")


In [5]:
from openai import OpenAI
client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)


In [18]:
json_completion = client.beta.chat.completions.parse(
    model="gemma2:9b-instruct-q8_0",

    response_format= PoeticResponses,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write about the wonders of GPU computing. Give 5 different versions"}
    ],
    temperature=0.02,
    )
PoeticResponses.model_validate(json_completion.choices[0].message.parsed)

PoeticResponses(responses=[PoeticResponse(User_Instruction='Write about the wonders of GPU computing.', Poem="Within silicon hearts, a million threads ignite, \nNo longer bound by CPU's single might. \nThe GPU awakes, a parallel dream, \nWhere tasks divide and conquer, it would seem. \nFrom rendering worlds to learning AI's art,\nGPU computing sets the future apart.", Limerick='There once was a GPU so grand, \nIt crunched numbers across the land. \nWith threads in a throng, \nIt made calculations strong, \nAnd left CPUs feeling quite unmanned.', Rap_Lyric="Yo, check it, the GPU's on fire, \nParallel processing, takin' it higher. \nMillions of cores, a computational beast,\nLeavin' CPUs in the dust, at least. \nFrom games to science, it's makin' waves,\nThe future is here, no time for caves."), PoeticResponse(User_Instruction='Write about the wonders of GPU computing.', Poem="A symphony of silicon, a chorus bright,\nGPU's dance with data, day and night. \nEach core a dancer, swift and k

In [19]:
json_completion.choices[0].message.parsed.model_dump()

{'responses': [{'User_Instruction': 'Write about the wonders of GPU computing.',
   'Poem': "Within silicon hearts, a million threads ignite, \nNo longer bound by CPU's single might. \nThe GPU awakes, a parallel dream, \nWhere tasks divide and conquer, it would seem. \nFrom rendering worlds to learning AI's art,\nGPU computing sets the future apart.",
   'Limerick': 'There once was a GPU so grand, \nIt crunched numbers across the land. \nWith threads in a throng, \nIt made calculations strong, \nAnd left CPUs feeling quite unmanned.',
   'Rap_Lyric': "Yo, check it, the GPU's on fire, \nParallel processing, takin' it higher. \nMillions of cores, a computational beast,\nLeavin' CPUs in the dust, at least. \nFrom games to science, it's makin' waves,\nThe future is here, no time for caves."},
  {'User_Instruction': 'Write about the wonders of GPU computing.',
   'Poem': "A symphony of silicon, a chorus bright,\nGPU's dance with data, day and night. \nEach core a dancer, swift and keen,\nWe